### Labor force participation in times of COVID-19 ###

First, let's import our packages necessary to download the data, work with it, create plots and run our exercises:

In [1]:
# FOR IMPORTING DATA

import requests
import json
import csv
import datetime

# FOR API
from dotenv import load_dotenv
import os
import pandas as pd
from typing import List, Union, Optional

# FOR CHARTS
import matplotlib.pyplot as plt
%matplotlib inline

First, let's set our API Key to be used for BLS, which is saved on a separate .txt file for security purposes

In [2]:
txt_file = open("BLS_API_KEY.txt")
apikey = txt_file.read()

Using the knowledge learned in class, we copy and adapt a function to download data from the BLS API:

In [3]:
def request_for_series(
        series_ids: List[str], 
        startyear: Union[int,str], 
        endyear: Union[int,str],
        apikey:Optional[str]=None,
    ):
    """
    Request data for all ``series_ids`` between ``startyear`` and ``endyear``
    
    Parameters
    ----------
    series_ids: List[str]
        A list of all BLS series IDs for which to request data
    
    startyear, endyear: Union[int,str]
        Starting and ending years for period of data. All intervals between
        these two years (inclusive) will be reported
    
    apikey: Optional[str]
        A registration or API key to enable more extensive use of the
        api and more detailed results
    
    Returns
    -------
    responses: List[requests.Response]
        A list of `Response` objects from the requests library
    
    Notes
    -----
    The BLS API only allows 25 series to be requested in a single call to the API
    This function allows an arbitrary number of series. The function first checks how 
    many series_ids are requested, and then makes two recursive calls to this function:
    
    (1) the first 25 series IDs are fetched and 
    (2) the rest of the series ids. 
    
    If the second request contains more than 25 series, another split is made and a pair of 
    recursive function calls are issued.
    
    Also note that the response objects from ``requests`` are not processed or validated
    in any way -- this is up to the caller of this routine.
    
    Finally, if an apikey is given, then a catalog of series metadata will be requested
    and returned from this function. This is necessary for getting the metadata DataFrame
    from the functions ``make_dfs_from_series``, ``unpack_response``, and 
    ``unpack_all_responses`` functions below.
    """
    n_series = len(series_ids)
    if n_series > 25:  #for more than 25, you work on this
        parts = []
        # make common keyword arguments so we don't have to type twice below
        kw = dict(endyear=endyear, startyear=startyear, apikey=apikey)
        parts.extend(request_for_series(series_ids[:25], **kw))
        parts.extend(request_for_series(series_ids[25:], **kw))
        return parts
    else:
        headers = {'Content-type': 'application/json'}
        params = {
            "seriesid": series_ids, 
            "startyear":startyear, 
            "endyear": endyear,
        }
        if apikey is not None:
            params["catalog"] = True
            params["registrationkey"] = apikey
        
        # convert params dictionary to json string
        data = json.dumps(params)
        p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
        return [p]

This function is used to validate the search through the website (i.e. to see if it works):

In [4]:
def check_response(res: requests.Response):
    """
    Check a response from the BLS API for success
    
    Parameters
    ----------
    res: requests.Response
        The requests object returned from iteracting with BLS API
        
    Notes
    -----
    Right now we just check for success at the http protocol level
    and don't do any checking specific to the BLS api
    """
    code = res.status_code
    if code > 299:
        raise ValueError(f"Response error with code {code}")

This function is to transfer the data into dataframes:

In [5]:
def make_dfs_from_series(series_results):
    """
    Unpack a series response object into data and metadata pandas objects
    
    Parameters
    ----------
    series_results: dict
        A dictionary returned from the `timeseries/data` endpoint of the
        BLS api. An example object for this parameter would be found at
        ``res.json()["Results"]["series"][0]`` where ``res`` is the 
        ``requests.Response`` obtained from interacting with the API endpoint.
    
    Returns
    -------
    data: pd.DataFrame
        A pandas DataFrame containing the actual observations of the data series
    
    metadata: Optional[pd.Series]
        If the ``"catgalog"`` key exists in ``series_results``, then ``metadata``
        is a pandas Series containing the catalog information. If ``"catalog"``
        is found, then this is None    
    """
    #extract series ID and store as a variable
    series_id = series_results["seriesID"]
    
    #next, we create a panda dataframe, reading a list of dict
    #we attach ID as an additional column
    data = pd.DataFrame(series_results["data"]).assign(series_id=series_id)
    
    #if catalog was attached, we create a series
    #if not, we don't
    if "catalog" in series_results:
        metadata = pd.Series(series_results["catalog"])
    else:
        metadata = None
    
    return data, metadata

These two functions will unpack the data:

In [6]:
def unpack_response(res: requests.Response):
    """
    Unpack the response for requesting one or more timeseries 
    from the BLS api
    
    Parameters
    ----------
    res: requests.Response
        The object returned from interacting with the ``timeseries/data``
        BLS API endpoint via the reuqests library
    
    Returns
    -------
    datasets: List[Tuple[pd.DataFrame, pd.Series]]
        For each BLS series contained in ``res``, a tuple with the 
        timeseries observations and series metadata will be returned.
        The observations are a pandas DataFrame and the metadata is a
        pandas Series. These pairs of (data, metadata) are returned
        in a list
    
    See Also
    --------
    See ``make_dfs_from_series`` for more information on content
    of output.
    """
    #compute the json form of our response
    js = res.json()
    #now we map the results and series as above when we created data and meta
    
    return list(map(make_dfs_from_series, js["Results"]["series"]))


def unpack_all_responses(all_res: List[requests.Response]):
    """
    Given a list of responses from the BLS API, extract and 
    return all data and metadata
    
    Parameters
    ----------
    all_res: List[requests.Response])
        Each item in this list is the result of using ``requests`` to 
        fetch data from ``timeseries/data`` endpoint of the BLS API.
        
    Returns
    -------
    data: pd.DataFrame
        A pandas DataFrame containing all timeseries observations included
        in any of the responses in ``all_res``
    
    metadata: pd.DataFrame
        Detailed metadata about each series, if such metadata exists in the
        response objects
    
    See Also
    --------
    See ``unpack_response`` and ``make_dfs_from_series`` functions
        
    """
    #make unpacked an empty list
    unpacked = []
    
    #then use a for function to unpack all of them (list)
    for res in all_res:
        unpacked.extend(unpack_response(res))
    
    data_dfs, metadata_series = list(zip(*unpacked))
    
    #stack all data one on the top of the other
    data = pd.concat(data_dfs, ignore_index=True)
    
    #add the metadata and stack them as columns
    metadata = pd.concat([x for x in metadata_series if x is not None], axis=1).T
    
    return data, metadata

Now, let's find all the series of interest.

If it ends with:

3. Seasonally adjustment unemployment rate. 
4. Seasonally adjusted unemployment level
5. Seasonally adjusted employment level
6. Seasonally adjusted labor force level
7. Seasonally adjusted employment-population ratio
8. Seasonally adjusted labor force participation ratio

We will use only 3 and 8 in our analysis. The first two numbers refer to the state from which the data is extracted.

Source: https://www.bls.gov/lau/

In [7]:
series_ids = [
    "LASST010000000000003", 
    "LASST010000000000008", 
    "LASST020000000000003", 
    "LASST020000000000008", 
    "LASST030000000000003",
    "LASST030000000000008", 
    "LASST040000000000003",
    "LASST040000000000008", 
    "LASST050000000000003", 
    "LASST050000000000008", 
    "LASST060000000000003", 
    "LASST060000000000008", 
    "LASST070000000000003", 
    "LASST070000000000008", 
    "LASST080000000000003", 
    "LASST080000000000008", 
    "LASST090000000000003",
    "LASST090000000000008", 
    "LASST100000000000003",
    "LASST100000000000008", 
    "LASST110000000000003", 
    "LASST110000000000008", 
    "LASST120000000000003", 
    "LASST120000000000008", 
    "LASST130000000000003", 
    "LASST130000000000008", 
    "LASST140000000000003", 
    "LASST140000000000008", 
    "LASST150000000000003", 
    "LASST150000000000008", 
    "LASST160000000000003", 
    "LASST160000000000008", 
    "LASST170000000000003", 
    "LASST170000000000008", 
    "LASST180000000000003", 
    "LASST180000000000008", 
    "LASST190000000000003", 
    "LASST190000000000008",
    "LASST200000000000003", 
    "LASST200000000000008", 
    "LASST210000000000003", 
    "LASST210000000000008", 
    "LASST220000000000003", 
    "LASST220000000000008", 
    "LASST230000000000003", 
    "LASST230000000000008", 
    "LASST240000000000003", 
    "LASST240000000000008", 
    "LASST250000000000003", 
    "LASST250000000000008", 
    "LASST260000000000003", 
    "LASST260000000000008", 
    "LASST270000000000003", 
    "LASST270000000000008", 
    "LASST280000000000003", 
    "LASST280000000000008", 
    "LASST290000000000003", 
    "LASST290000000000008", 
    "LASST300000000000003", 
    "LASST300000000000008", 
    "LASST310000000000003",
    "LASST310000000000008", 
    "LASST320000000000003", 
    "LASST320000000000008", 
    "LASST330000000000003", 
    "LASST330000000000008",
    "LASST340000000000003", 
    "LASST340000000000008", 
    "LASST350000000000003",
    "LASST350000000000008",  
    "LASST360000000000003", 
    "LASST360000000000008", 
    "LASST370000000000003", 
    "LASST380000000000008", 
    "LASST370000000000003", 
    "LASST380000000000008", 
    "LASST370000000000003", 
    "LASST380000000000008", 
    "LASST390000000000003", 
    "LASST390000000000008",    
    "LASST400000000000003", 
    "LASST400000000000008", 
    "LASST410000000000003", 
    "LASST410000000000008", 
    "LASST420000000000003", 
    "LASST420000000000008", 
    "LASST430000000000003", 
    "LASST430000000000008", 
    "LASST440000000000003", 
    "LASST440000000000008", 
    "LASST450000000000003", 
    "LASST450000000000008", 
    "LASST460000000000003", 
    "LASST460000000000008", 
    "LASST470000000000003", 
    "LASST470000000000008", 
    "LASST480000000000003", 
    "LASST480000000000008", 
    "LASST490000000000003", 
    "LASST490000000000008", 
    "LASST500000000000003", 
    "LASST500000000000008",
    "LASST510000000000003", 
    "LASST510000000000008",
    "LASST520000000000003",
    "LASST520000000000008",
    "LASST530000000000003", 
    "LASST530000000000008", 
    "LASST540000000000003", 
    "LASST540000000000008",     
    "LASST550000000000003", 
    "LASST550000000000008", 
    "LASST560000000000003", 
    "LASST560000000000008",
]

# make requests, get responses
responses1 = request_for_series(series_ids, "1976", "1995", apikey=apikey)

# validate the responses
[check_response(r) for r in responses1]

# extract data and metadata from responses
df1, metadata1 = unpack_all_responses(responses1)


In [8]:
# pull data for date range: 1996 - 2015
responses2 = request_for_series(series_ids, "1996", "2015", apikey=apikey)
df2, metadata2 = unpack_all_responses(responses2)

In [9]:
# pull data for date range: 2016 - 2021
responses3 = request_for_series(series_ids, "2016", "2021", apikey=apikey)
df3, metadata3 = unpack_all_responses(responses3)

This project will also incorporate data related to the dates when states announced that Pandemic Unemployment Assistance benefits (PUA)

will expire (announce_ui) in additon to the actual date when PUA ended (expired_ui). 

In [ ]:
announce_url = "https://raw.githubusercontent.com/vgm236/ferrante_moreira_data_project/main/Data/announced_dates_ui_expiration.csv"
announce_ui = pd.read_csv(announce_url)

expired_url = "https://raw.githubusercontent.com/vgm236/ferrante_moreira_data_project/main/Data/effective_dates_ui_expiration.csv"
expired_ui = pd.read_csv(expired_url)

Since the data was pulled in three batches, we will now combine to one large dataframe for unemployment and labor force participation values 

and one larger dataframe for metadata

In [189]:
# create single dataframe for df1, df2, and df3
df = pd.concat([df1, df2, df3])

df = df.sort_values(by=["year"]) 

df["year"].unique()

array(['1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021'], dtype=object)

In [191]:
# create single dataframe for metadata1, metadata2, metadata3
metadata = pd.concat([metadata1, metadata2, metadata3])

Since the Metadata dataframe is composed of both unemployment rate and labor force participation, we will separate accordingly 

In [291]:
# Dataframe including only measure data types for unemployment rate
df_ur = metadata.query("measure_data_type == 'unemployment rate'")

# Dataframe including only measure data types for labor force participation
df_lfp = metadata.query("measure_data_type == 'labor force participation rate'")

Now that we have the dataframes seperated by unemployment rate and labor force participation, we can map values from the df 

dataframe to the df_ur and df_lfp dataframes using series_ids

In [274]:
# Merge df on df_ur by series id to get each states unemployment rate
df_ur_values = df_ur.merge(df, on="series_id", how="left")

In [276]:
# Merge df on df_lfp by series id to get each states labor force participation rate
df_lfp_values = df_lfp.merge(df, on="series_id", how="left")

In [193]:
# merge unemployment announcement date and expiration date to single dataframe 
ui_dates = announce_ui.merge(expired_ui, on="state_abrev", how="left")

# change to datetime 
ui_dates["effe_date"] = pd.to_datetime(ui_dates["effe_date"])
ui_dates["annc_date"] = pd.to_datetime(ui_dates["annc_date"])

# change state_abrev to area for future merging simplicity
state = {"state_abrev" : "area"}
ui_dates = ui_dates.rename(columns=state)

In [251]:
# change "area" values to abbrevations to allow for merging ui_dates  
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

df_ur_values["area"] = df_ur_values["area"].replace(us_state_to_abbrev)

df_lfp_values["area"] = df_lfp_values["area"].replace(us_state_to_abbrev)

In [290]:
# Data Preparation for analyses

# Create smaller DataFrames for UR and LFP with only necessary columns
df_ur_small = df_ur_values[["area", "year", "period", "value"]]
df_lfp_small = df_lfp_values[["area", "year", "period", "value"]]

# Remove the M from period to create an integer for the month, then create a new column which is the date (Year-Month-Day)
df_ur_small["month"] = df_ur_small["period"].str.strip("M").astype(int) 
df_ur_small["date"] = pd.to_datetime(df_ur_small[["year", "month"]].assign(DAY=1))

df_lfp_small["month"] = df_lfp_small["period"].str.strip("M").astype(int)
df_lfp_small["date"] = pd.to_datetime(df_lfp_small[["year", "month"]].assign(DAY=1))

# Re-label values in both DataFrames so it is clear which refers to UR and LFP once merged
ur_value_name = {"value" : "unemployment_rate"}
lfp_value_name = {"value" : "labor_force_participation"}

df_ur_small = df_ur_small.rename(columns=ur_value_name)
df_lfp_small = df_lfp_small.rename(columns=lfp_value_name)

# Drop uncessary columns
df_ur_small = df_ur_small[["area", "unemployment_rate", "date"]]
df_lfp_small = df_lfp_small[["area", "labor_force_participation", "date"]]

# Merge the two Dataframe on area and year
df_ur_lfp = df_ur_small.merge(
    df_lfp_small, how='left', 
    left_on=['area', 'date'], 
    right_on=['area', 'date']
    ).drop_duplicates(subset=["area", "date"])

# Turn UR and LFP values to float 
df_ur_lfp["unemployment_rate"] = df_ur_lfp["unemployment_rate"].astype(float)
df_ur_lfp["labor_force_participation"] = df_ur_lfp["labor_force_participation"].astype(float)

# Reshape data for analyses
df_ur_lfp = df_ur_lfp.pivot(
    index="date", 
    columns="area", 
    values=["unemployment_rate", "labor_force_participation"])

df_ur_lfp

C:\Users\alexa\AppData\Local\Temp/ipykernel_18540/2261253580.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ur_small["month"] = df_ur_small["period"].str.strip("M").astype(int)
C:\Users\alexa\AppData\Local\Temp/ipykernel_18540/2261253580.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ur_small["date"] = pd.to_datetime(df_ur_small[["year", "month"]].assign(DAY=1))
C:\Users\alexa\AppData\Local\Temp/ipykernel_18540/2261253580.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy 

unemployment_rate                                              \
area                 Alabama Alaska Arizona Arkansas California Colorado   
date                                                                       
1976-01-01               6.6    7.1    10.2      7.3        9.2      5.8   
1976-02-01               6.6    7.1    10.2      7.3        9.2      5.7   
1976-03-01               6.6    7.0    10.1      7.3        9.1      5.7   
1976-04-01               6.5    6.9    10.0      7.2        9.1      5.6   
1976-05-01               6.4    6.9     9.8      7.1        9.0      5.6   
...                      ...    ...     ...      ...        ...      ...   
2021-06-01               3.3    6.6     6.8      4.4        7.6      6.2   
2021-07-01               3.2    6.6     6.6      4.3        7.6      6.1   
2021-08-01               3.1    6.4     6.2      4.2        7.5      5.9   
2021-09-01               3.1    6.3     5.7      4.0        7.5      5.6   
2021-10-01               3.1    6.1     5.2      3.7        7.3      5.4   

                                                              ...  \
area       Connecticut Delaware District of Columbia Florida  ...   
date                                                          ...   
1976-01-01         9.8      8.0                  8.7     9.7  ...   
1976-02-01         9.8      8.0                  8.7     9.7  ...   
1976-03-01         9.8      8.0                  8.6     9.6  ...   
1976-04-01         9.6      8.1                  8.5     9.5  ...   
1976-05-01         9.4      8.3                  8.4     9.3  ...   
...                ...      ...                  ...     ...  ...   
2021-06-01         7.7      5.8                  7.0     5.0  ...   
2021-07-01         7.3      5.6                  6.7     5.1  ...   
2021-08-01         7.2      5.4                  6.7     5.0  ...   
2021-09-01         6.8      5.4                  6.6     4.8  ...   
2021-10-01         6.4      5.3                  6.3     4.6  ...   

           labor_force_participation                                         \
area                    South Dakota Tennessee Texas  Utah Vermont Virginia   
date                                                                          
1976-01-01                      64.6      59.6  63.8  62.8    63.6     65.9   
1976-02-01                      64.4      59.4  63.5  62.6    63.4     65.8   
1976-03-01                      64.4      59.3  63.4  62.5    63.2     65.6   
1976-04-01                      64.3      59.3  63.5  62.5    63.5     65.5   
1976-05-01                      64.3      59.4  63.6  62.6    63.6     65.4   
...                              ...       ...   ...   ...     ...      ...   
2021-06-01                      68.7      60.7  62.2  67.4    60.6     62.7   
2021-07-01                      68.7      60.6  62.3  67.8    60.9     62.8   
2021-08-01                      68.7      60.6  62.3  67.9    61.2     62.9   
2021-09-01                      68.7      60.3  62.4  67.9    61.3     62.9   
2021-10-01                      68.7      60.1  62.5  67.9    61.3     63.0   

                                                       
area       Washington West Virginia Wisconsin Wyoming  
date                                                   
1976-01-01       60.9          52.5      65.6    65.3  
1976-02-01       60.7          52.4      65.5    64.9  
1976-03-01       60.6          52.3      65.4    64.7  
1976-04-01       60.6          52.3      65.4    64.9  
1976-05-01       60.7          52.2      65.5    65.2  
...               ...           ...       ...     ...  
2021-06-01       63.1          55.2      66.3    64.8  
2021-07-01       63.3          55.2      66.4    64.6  
2021-08-01       63.5          55.2      66.5    64.3  
2021-09-01       63.5          55.2      66.5    64.1  
2021-10-01       63.6          55.2      66.4    63.9  

[550 rows x 100 columns]